## Intialize the environment

In [ ]:
%reset -f

import os
from shutil import which
import glob
import  pandas as pd

import tifffile
import numpy as np
import matplotlib.pyplot as plt

plt.ion()

import matplotlib
matplotlib.use('TkAgg')  # Force Tk backend

%matplotlib tk

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Load the separated channels into numpy arrays


In [ ]:
plt.close("all")

# Load the tiff file and save the images as numpy arrays
Current_folder = os.getcwd()

IMAGE_PATH = "Image-after"

full_path_to_load = f"/media/tatsatb/Images/{IMAGE_PATH}/{IMAGE_PATH}.tif"


Parent_Dir = os.path.dirname(full_path_to_load)
Path_to_Save = os.path.join(Parent_Dir, "Image_saved_as_MATRIX")
os.makedirs(Path_to_Save, exist_ok=True)

tiff_file = full_path_to_load
info = tifffile.TiffFile(tiff_file)

n_images = len(info.pages)
cols = info.pages[0].shape[1]
rows = info.pages[0].shape[0]


n_frames = int(n_images / 2)


## Load the image, select the frames, and save separate channels

In [ ]:

I_tmp = np.zeros((rows, cols, n_images), dtype=np.uint16)

initial = 1


full_path_red_channel_data = os.path.join(Path_to_Save, 'Red_channel.npy')
full_path_DIC_channel_data = os.path.join(Path_to_Save, 'DIC_channel.npy')


for i in range(0, n_images, 1):
    tmp = tifffile.imread(tiff_file, key=i)
    I_tmp[:, :, i] = tmp


DIC_channel = I_tmp[:, :, 1::2]
Red_channel = I_tmp[:, :, 0::2]


np.save(full_path_red_channel_data, Red_channel)
np.save(full_path_DIC_channel_data, DIC_channel)

plt.figure(figsize=(4, 4))
ax1 = plt.gca()
plt.imshow(Red_channel[:, :, 0], cmap='viridis')
ax1.grid(False)
plt.colorbar()
plt.title('Channel Red - First Frame')
plt.tight_layout()
plt.show()


plt.figure(figsize=(4, 4))
ax3 = plt.gca()
plt.imshow(DIC_channel[:, :, 0], cmap='viridis')
ax3.grid(False)
plt.colorbar()
plt.title('Channel DIC - First Frame')
plt.show()



## View the Images, select a polygon that encompasses the cell, and create a mask

In [ ]:
from roipoly import RoiPoly
from matplotlib.path import Path
plt.ion()


def enhance_contrast(frame, low_in=2, high_in=98):

    # Convert to float for processing
    frame_float = frame.astype(float)

    # Calculate percentiles for contrast stretching
    low_val = np.percentile(frame_float, low_in)
    high_val = np.percentile(frame_float, high_in)

    # Clip and stretch contrast
    adjusted = np.clip(frame_float, low_val, high_val)
    adjusted = ((adjusted - low_val) * 255 / (high_val - low_val))

    # Ensure uint8 output
    return np.clip(adjusted, 0, 255).astype(np.uint8)


def process_single_channel(frame, frame_num):
    # Enhance contrast for display
    frame_display = enhance_contrast(frame)

    attempt = 1
    while True:
        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(111)
        ax.grid(False)
        ax.imshow(frame_display, cmap='gray')
        ax.set_title(f'Draw ROI - Frame {frame_num+1} - Attempt {attempt}')

        try:
            roi = RoiPoly(fig=fig, ax=ax, color='r')
            plt.show(block=True)

            # Verify ROI points exist
            if hasattr(roi, 'x') and len(roi.x) > 2:
                ny, nx = frame.shape
                poly_verts = [(roi.x[i], roi.y[i]) for i in range(len(roi.x))]
                x, y = np.meshgrid(np.arange(nx), np.arange(ny))
                xy = np.vstack((x.flatten(), y.flatten())).T

                # Create path for mask
                path = Path(poly_verts)
                mask = path.contains_points(xy).reshape(frame.shape)

                processed = frame.copy()
                processed[~mask] = 0
                plt.close(fig)
                return processed, mask
            else:
                print("Invalid ROI - please try again")
                plt.close(fig)
                attempt += 1
                continue

        except KeyboardInterrupt:
            print("Process interrupted by user. Exiting...")
            plt.close(fig)
            return None, None

        except Exception as e:
            print(f"ROI Error: {str(e)}")
            plt.close(fig)
            attempt += 1
            continue


def process_all_frames(single_channel, masks=None):
    processed = np.zeros_like(single_channel)
    all_masks = np.zeros_like(single_channel, dtype=bool)

    for i in range(0,n_frames):
        processed[:,:,i], mask = process_single_channel(single_channel[:,:,i].copy(), i)
        all_masks[:,:,i] = mask
    return processed, all_masks


# Process frames


processed_red_channel, red_masks = process_all_frames(Red_channel)



## Threshold and binarize the images

In [ ]:
from scipy.signal import convolve2d
from matplotlib.widgets import Cursor
plt.rc("axes",grid=False)

smoothed_image_red = np.zeros_like(processed_red_channel)

kernel = np.ones((3, 3)) / 9.0

for channel in range(smoothed_image_red.shape[2]):
    smoothed_image_red[:, :, channel] = convolve2d(processed_red_channel[:, :, channel], kernel, mode='same', boundary='fill', fillvalue=0)

threshold = 2600

thresholded_combined = np.where(smoothed_image_red > threshold, 1, 0)

plt.figure(figsize=(12, 4))



ax1 = plt.subplot(221)
im1 = plt.imshow(processed_red_channel[:, :, 0], cmap='viridis')
plt.title('Processed Red Channel - First Frame')


ax2 = plt.subplot(222)
im2 = plt.imshow(smoothed_image_red[:, :, 0], cmap='viridis')
plt.title('Smoothed Red Channel - First Frame')


ax3 = plt.subplot(223)
im3 = plt.imshow(thresholded_combined[:, :, 0], cmap='viridis')
plt.title('Thresholded Red Channel - First Frame')


ax4 = plt.subplot(224)
im4 = plt.imshow(enhance_contrast(DIC_channel[:, :, 0]), cmap='gray')
plt.title('DIC Channel - First Frame')


plt.tight_layout()
plt.show(block=True)
plt.close()



## Process the binary images with morphological operations 

In [ ]:
from scipy import ndimage
from skimage.morphology import binary_erosion, disk, binary_dilation
from skimage.filters import median


eroded = np.zeros_like(thresholded_combined)
despecked = np.zeros_like(thresholded_combined)
filled = np.zeros_like(thresholded_combined)
dilated = np.zeros_like(thresholded_combined)
mask_image_update = np.zeros_like(thresholded_combined)


for frame in range(thresholded_combined.shape[2]):

    # Fill holes in the image
    filled[:, :, frame] = ndimage.binary_fill_holes(thresholded_combined[:, :, frame])

    # Erode the image
    eroded[:, :, frame] = binary_erosion(filled[:, :, frame], disk(2))

    # Despeckle the image using median filter
    despecked[:, :, frame] = median(eroded[:, :, frame], disk(1))

    # Dilate the image
    dilated[:, :, frame] = ndimage.binary_dilation(despecked[:, :, frame], disk(2))

    # Update mask image
    mask_image_update[:, :, frame] = dilated[:, :, frame]


fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.imshow(mask_image_update[:,:,0], cmap='viridis')

## Save the thresholded image as final mask of images

In [ ]:

full_path_mask = os.path.join(Path_to_Save, 'Thesholded_mask.npy')
np.save(full_path_mask, mask_image_update)

mask_image_final_8bit = (mask_image_update * 255).astype(np.uint8)

mask_save_path = os.path.join(Path_to_Save, 'mask_8bit.tif')
tifffile.imwrite(mask_save_path, mask_image_final_8bit.transpose(2,0,1))


## If you need to load the saved mask later, use:


In [ ]:
full_path_mask = os.path.join(Path_to_Save, 'Thresholded_mask.npy')
mask_save_path = os.path.join(Path_to_Save, 'mask_8bit.tif')

mask_image_load = tifffile.imread(mask_save_path)

mask_image_reshape = mask_image_load.transpose(1, 2, 0)
mask_image_final = mask_image_reshape.astype(np.int64)/255


## Find the centroid of the mask

In [ ]:
# Find the centroid of the mask

from skimage.measure import regionprops, label

def find_centroids(binary_image):
    """Find centroids in all frames of a binary 3D image."""
    num_frames = binary_image.shape[2]
    all_centroids = np.zeros((num_frames, 2))
    
    for frame in range(num_frames):
        # Get current frame
        current_frame = binary_image[:,:,frame]
        
        # Label connected components
        labeled_frame = label(current_frame)
        
        # Get properties of labeled regions
        regions = regionprops(labeled_frame)
        
        # Extract centroids for the largest region in this frame
        if regions:
            largest_region = max(regions, key=lambda r: r.area)
            centroid = largest_region.centroid
            all_centroids[frame] = (centroid[1], centroid[0])  # (x, y) coordinates
    
    return all_centroids

# Apply to thresholded image
centroids = find_centroids(mask_image_final)

# Optional: Plot results for verification
plt.figure(figsize=(8,8))
plt.imshow(mask_image_final[:,:,0], cmap='gray')
plt.plot(centroids[0, 0], centroids[0, 1], 'r+')
plt.title('Detected Centroids (Frame 0)')
plt.show()



## Plot more results for further verification

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(mask_image_final[:,:,4], cmap='gray')
plt.plot(centroids[4, 0], centroids[4, 1], 'r+')
plt.title('Detected Centroids')
plt.show()

In [ ]:
print(centroids.shape)

## Save the centroid data



In [ ]:
import pandas as pd

# Save the centroid data to an Excel file
centroid_df = pd.DataFrame(centroids, columns=['X', 'Y'])
excel_path = os.path.join(Path_to_Save, 'centroids.xlsx')
centroid_df.to_excel(excel_path, index=False)

print(f"Centroid data saved to {excel_path}")

## Load the centroid data and do necessary calculations

In [ ]:
excel_path = os.path.join(Path_to_Save, 'centroids.xlsx')
centroid_df_loaded = pd.read_excel(excel_path)
centroids = centroid_df_loaded.to_numpy()


pixel_width = 0.2767553 # pixel width 
centroids_adjusted = centroids - centroids[0]
centroids_adjusted_scaled = centroids_adjusted * pixel_width

Path_to_centroid_Save = os.path.join(Parent_Dir, "..", "Centroids")
excel_path_centroid = os.path.join(Path_to_centroid_Save, full_path_to_load.split('/')[-1].replace('.tif', '.xlsx'))

os.makedirs(Path_to_centroid_Save, exist_ok=True)

centroid_adjusted_scaled_df = pd.DataFrame(centroids_adjusted_scaled, columns=['X', 'Y'])
centroid_adjusted_scaled_df.to_excel(excel_path_centroid, index=False)

print(f"Centroid data saved to {excel_path_centroid}")


distances = np.sqrt(np.sum(np.diff(centroids_adjusted_scaled, axis=0)**2, axis=1))

time_interval = 2 # time interval in minutes [adjust if required in seconds]
velocity = distances/time_interval


average_velocity = np.mean(velocity)

Path_to_velocity_Save = os.path.join(Parent_Dir, "..", "Velocities")
os.makedirs(Path_to_velocity_Save, exist_ok=True)

excel_path_velocity = os.path.join(Path_to_velocity_Save, full_path_to_load.split('/')[-1].replace('.tif', '.xlsx'))

average_velocity_df = pd.DataFrame([average_velocity], columns=['Average Velocity'])
average_velocity_df.to_excel(excel_path_velocity, index=False)

print(f"Velocity data saved to {excel_path_velocity}")

## Plot the centroid data trajectory

In [ ]:

Path_to_centroid_Save = os.path.join(Parent_Dir, "..", "Centroids")
excel_path_centroid = os.path.join(Path_to_centroid_Save, full_path_to_load.split('/')[-1].replace('.tif', '.xlsx'))

# Load all the centroid data files from the specified directory
centroid_files_before = sorted(glob.glob(os.path.join(Path_to_centroid_Save, '*-before.xlsx')))
centroid_files_after = sorted(glob.glob(os.path.join(Path_to_centroid_Save,  '*-after.xlsx')))


# Initialize an empty DataFrame to store all the stacked data
stacked_centroid_before_df = pd.DataFrame()
stacked_centroid_after_df = pd.DataFrame()

# Loop through each file and stack the data
for file in centroid_files_before:
    centroid_df_before = pd.read_excel(file)
    stacked_centroid_before_df = pd.concat([stacked_centroid_before_df, centroid_df_before[['X', 'Y']]], axis=1)

# print(stacked_centroid_before_df)

stacked_centroid_before_df.to_excel(os.path.join(Parent_Dir, '..', 'stacked_centroids_before.xlsx'), index=False)

stacked_centroid_before_array = stacked_centroid_before_df.to_numpy()

plt.figure(figsize=(8,8))
colors = plt.cm.tab20.colors + plt.cm.tab20b.colors  # Combines two colormaps for more distinct colors

for i in range(0, stacked_centroid_before_array.shape[1], 2):
    color_idx = i // 2
    plt.plot(stacked_centroid_before_array[:, i], stacked_centroid_before_array[:, i + 1],
             color=colors[color_idx % len(colors)])
    
plt.legend()
plt.title('Before')
plt.axhline(0, ls = '--', color = 'k', alpha = 0.4, lw = 1)
plt.axvline(0, ls = '--', color = 'k', alpha = 0.4, lw = 1)
plt.axis([-250, 250, -250, 250])


for file in centroid_files_after:
    centroid_df_after = pd.read_excel(file)
    stacked_centroid_after_df = pd.concat([stacked_centroid_after_df, centroid_df_after[['X', 'Y']]], axis=1)

print(stacked_centroid_before_df)
print(stacked_centroid_after_df)

stacked_centroid_after_df.to_excel(os.path.join(Parent_Dir, '..', 'stacked_centroids_after.xlsx'), index=False)

stacked_centroid_after_array = stacked_centroid_after_df.to_numpy()

plt.figure(figsize=(8,8))
colors = plt.cm.tab20.colors + plt.cm.tab20b.colors
for i in range(0, stacked_centroid_after_array.shape[1], 2):
    color_idx = i // 2
    plt.plot(stacked_centroid_after_array[:, i], stacked_centroid_after_array[:, i + 1],
             color=colors[color_idx % len(colors)])
plt.legend()
plt.title('After')
plt.axhline(0, ls = '--', color = 'k', alpha = 0.4, lw = 1)
plt.axvline(0, ls = '--', color = 'k', alpha = 0.4, lw = 1)
plt.axis([-210, 210, -210, 210])


## Plot the velocity data

In [ ]:
Path_to_velocity_Save = os.path.join(Parent_Dir, "..", "Velocities")

# Load all the velocity data files from the specified directory
velocity_files_before = sorted(glob.glob(os.path.join(Path_to_velocity_Save, '*-before.xlsx')))
velocity_files_after = sorted(glob.glob(os.path.join(Path_to_velocity_Save, '*-after.xlsx')))

velocity_before_df = pd.DataFrame()
velocity_after_df = pd.DataFrame()

# Loop through each file and stack the data
for file in velocity_files_before:
    velocity_df_before = pd.read_excel(file)
    velocity_df_before['Filename'] = os.path.basename(file)
    velocity_before_df = pd.concat([velocity_before_df, velocity_df_before], axis=0)

velocity_before_df.to_excel(os.path.join(Parent_Dir, "..", "Velocities", "1_Combined_Before.xlsx"), index=False)

for file in velocity_files_after:
    velocity_df_after = pd.read_excel(file)
    velocity_df_after['Filename'] = os.path.basename(file)
    velocity_after_df = pd.concat([velocity_after_df, velocity_df_after], axis=0)


velocity_after_df.to_excel(os.path.join(Parent_Dir, "..", "Velocities", "1_Combined_After.xlsx"), index=False)


In [ ]:
import seaborn as sns
sns.set_style("ticks", {'axes.grid' : False})

plt.figure(figsize=(6, 8))

# Create DataFrame with paired data
data = pd.DataFrame({
    'Velocity': pd.concat([velocity_before_df['Average Velocity'], velocity_after_df['Average Velocity']]),
    'Condition': ['Before'] * len(velocity_before_df) + ['After'] * len(velocity_after_df)
})

data_visible = pd.DataFrame({
    'Velocity': pd.concat([velocity_before_df['Average Velocity'], velocity_after_df['Average Velocity']]),
    'Condition': ['Before'] * len(velocity_before_df) + ['After'] * len(velocity_after_df),
    'Plotting': ['Visible'] * (len(velocity_before_df) + len(velocity_after_df))
})

data_invisible = pd.DataFrame({
    'Velocity': pd.concat([velocity_before_df['Average Velocity'], velocity_after_df['Average Velocity']]),
    'Condition': ['Before'] * len(velocity_before_df) + ['After'] * len(velocity_after_df),
    'Plotting': ['Invisible'] * (len(velocity_before_df) + len(velocity_after_df))
})

combined_data_visible = pd.concat([data_visible, data_invisible])
flierprops = {'marker': 'o', 'markerfacecolor': 'none', 'markeredgecolor': 'm', 'markersize': 9, 'linestyle': 'none'}




# Define custom colors
palet = sns.color_palette(["lightsteelblue", "white"])


# Create half violin plot
sns.violinplot(x='Condition', y='Velocity', hue='Plotting', data=combined_data_visible,
               split=True, inner=None, palette=palet, linewidth=0, alpha=0.59, legend=False)
sns.boxplot(x='Condition', y='Velocity', data=data, width=0.2, linewidth=1.8, palette="Paired", flierprops=flierprops)
sns.stripplot(x='Condition', y='Velocity', data=data, color='k', size=6, alpha=0.7, jitter=False)
sns.despine()

# Add lines connecting before and after points
before_points = velocity_before_df['Average Velocity'].values
after_points = velocity_after_df['Average Velocity'].values

for b, a in zip(before_points, after_points):
    plt.plot([0, 1], [b, a], color='gray', alpha=0.9, linestyle='solid', linewidth=1.2)

# plt.title('Velocity Distribution')
plt.xlabel("")
plt.ylabel('Velocity (µm/min)', fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
# plt.draw()
plt.show()





## Do the Statistical tests

In [ ]:
from scipy import stats

# Perform Mann-Whitney U test
statistic_mw, p_value_mw = stats.mannwhitneyu(
    velocity_before_df['Average Velocity'],
    velocity_after_df['Average Velocity'],
    alternative='two-sided'
)

# Perform Wilcoxon signed-rank test
statistic_w, p_value_w = stats.wilcoxon(
    velocity_before_df['Average Velocity'],
    velocity_after_df['Average Velocity'],
    alternative='two-sided'
)

print("Statistical Test Results:")
print("\nMann-Whitney U test:")
print(f"Statistic: {statistic_mw}")
print(f"P-value: {p_value_mw}")

print("\nWilcoxon signed-rank test:")
print(f"Statistic: {statistic_w}")
print(f"P-value: {p_value_w}")



## Plot all the velocity data together

In [ ]:
# Load all the velocity data files from the specified directory
base_directory = "/media/tatsatb/Data/"

Path_to_after_velocity_files = {}
Path_to_before_velocity_files = {}


for subdir in ['AX2', 'GEFB', 'GEFU', 'GEFX', 'GEFM']:
    Path_to_after_velocity_files[subdir] = os.path.join(base_directory, subdir, "Velocities", "1_Combined_After.xlsx")
    Path_to_before_velocity_files[subdir] = os.path.join(base_directory, subdir, "Velocities", "1_Combined_Before.xlsx")
    if not os.path.exists(Path_to_velocity_Save):
        print(f"Directory {Path_to_velocity_Save} does not exist.")
        continue


df_velocity_combined_after = pd.DataFrame()
df_velocity_combined_before = pd.DataFrame()

after_list = []
before_list = []

# Load each file and add as a column with the key as header
for key, path in Path_to_after_velocity_files.items():
    if os.path.exists(path):
        df = pd.read_excel(path)
        df["Group"] = key
        after_list.append(df[['Average Velocity', 'Group']])

    else:
        print(f"File not found: {path}")


for key, path in Path_to_before_velocity_files.items():
    if os.path.exists(path):
        df = pd.read_excel(path)
        df['Group'] = key
        before_list.append(df[['Average Velocity', 'Group']])
        
    else:
        print(f"File not found: {path}")
        continue


df_velocity_combined_after = pd.concat(after_list, ignore_index=True)
df_velocity_combined_before = pd.concat(before_list, ignore_index=True)



In [ ]:

# Combine before and after into one DataFrame for plotting

df_velocity_combined_before['Condition'] = 'Before'
df_velocity_combined_after['Condition'] = 'After'
df_all = pd.concat([df_velocity_combined_before, df_velocity_combined_after], ignore_index=True)


sns.set_style("ticks", {'axes.grid': False})


# Get all unique groups
all_groups = sorted(df_all['Group'].unique())

for group in all_groups:
    # Filter data for this group
    group_before = df_velocity_combined_before[df_velocity_combined_before['Group'] == group]
    group_after = df_velocity_combined_after[df_velocity_combined_after['Group'] == group]

    # Create DataFrame with paired data
    data = pd.DataFrame({
        'Velocity': pd.concat([group_before['Average Velocity'], group_after['Average Velocity']]),
        'Condition': ['Before'] * len(group_before) + ['After'] * len(group_after)
    })

    data_visible = pd.DataFrame({
        'Velocity': pd.concat([group_before['Average Velocity'], group_after['Average Velocity']]),
        'Condition': ['Before'] * len(group_before) + ['After'] * len(group_after),
        'Plotting': ['Visible'] * (len(group_before) + len(group_after))
    })

    data_invisible = pd.DataFrame({
        'Velocity': pd.concat([group_before['Average Velocity'], group_after['Average Velocity']]),
        'Condition': ['Before'] * len(group_before) + ['After'] * len(group_after),
        'Plotting': ['Invisible'] * (len(group_before) + len(group_after))
    })

    combined_data_visible = pd.concat([data_visible, data_invisible])
    flierprops = {'marker': 'o', 'markerfacecolor': 'none', 'markeredgecolor': 'm', 'markersize': 9, 'linestyle': 'none'}

    palet = sns.color_palette(["lightsteelblue", "white"])
    sns.set_style("ticks", {'axes.grid': False})

    plt.figure(figsize=(6, 8))
    sns.violinplot(x='Condition', y='Velocity', hue='Plotting', data=combined_data_visible,
                   split=True, inner=None, palette=palet, linewidth=0, alpha=0.59, legend=False)
    sns.boxplot(x='Condition', y='Velocity', data=data, width=0.2, linewidth=1.8, palette="Paired", flierprops=flierprops)
    sns.stripplot(x='Condition', y='Velocity', data=data, color='k', size=6, alpha=0.7, jitter=False)
    sns.despine()


    # Add lines connecting before and after points (if paired)
    before_points = group_before['Average Velocity'].values
    after_points = group_after['Average Velocity'].values
    for b, a in zip(before_points, after_points):
        plt.plot([0, 1], [b, a], color='gray', alpha=0.9, linestyle='solid', linewidth=1.2)


    plt.xlabel("")
    plt.ylabel('Velocity (µm/min)', fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.title(f'Velocity Distribution: {group}')
    plt.tight_layout()
    plt.show()







## Plot the velocity data: All groups together on same axis

In [ ]:
# Create DataFrame with paired data
data_combined = pd.concat([df_velocity_combined_before, df_velocity_combined_after])

data_combined_visible = data_combined.copy()
data_combined_visible['Plotting'] = 'Visible'


data_combined_invisible = data_combined.copy()
data_combined_invisible['Plotting'] = 'Invisible'


data_combined_final = pd.concat([data_combined_visible, data_combined_invisible])
data_combined_final["Group_Condition"] = data_combined_final['Group'] + '_' + data_combined_final['Condition']



# Define custom order for the x-axis
groups = data_combined_final['Group'].unique()
custom_order = []
for group in groups:
    custom_order.append(f"{group}_Before")
    custom_order.append(f"{group}_After")

flierprops_mod = {'marker': 'o', 'markerfacecolor': 'none', 'markeredgecolor': 'm', 'markersize': 9, 'linestyle': 'none'}

# Create the plot with the new x-axis variable and custom order
plt.figure(figsize=(12, 8))
sns.violinplot(x='Group_Condition', y='Average Velocity', hue='Plotting',
               data=data_combined_final, width=1.4, split=True, inner=None,  # Add quartile lines
               palette=palet, linewidth=0, alpha=0.59, legend=False,
               order=custom_order)  # Apply custom order

sns.boxplot(x='Group_Condition', y='Average Velocity', data=data_combined_final,
            width=0.22, linewidth=1.5, palette="Paired", flierprops=flierprops_mod,
            order=custom_order, whis=1.5)  # Add whiskers for 5th and 95th percentiles

sns.stripplot(x='Group_Condition', y='Average Velocity',data=data_combined_final,color='k', size=5, alpha=0.7, jitter=False,
             order=custom_order)

# Now add connecting lines for each group
for group in groups:
    before_data = data_combined_final[(data_combined_final['Group'] == group) &
                                     (data_combined_final['Condition'] == 'Before')]
    after_data = data_combined_final[(data_combined_final['Group'] == group) &
                                    (data_combined_final['Condition'] == 'After')]

    # Find x-positions for this group in the plot
    before_idx = custom_order.index(f"{group}_Before")
    after_idx = custom_order.index(f"{group}_After")

    # Match points (assuming they're in the same order)
    before_points = before_data['Average Velocity'].values
    after_points = after_data['Average Velocity'].values

    # Draw lines connecting each pair of points
    min_length = min(len(before_points), len(after_points))
    for i in range(min_length):
        plt.plot([before_idx, after_idx], [before_points[i], after_points[i]],
                 color='gray', alpha=0.5, linestyle='solid', linewidth=0.8)

sns.despine()
plt.ylim(-1, 20)
plt.xticks(rotation=15)  # Rotate labels for better readability
plt.tight_layout()  # Adjust layout to make room for rotated labels
plt.show()

plt.savefig('velocity_plot.svg', format='svg')
plt.savefig('velocity_plot.pdf', format='pdf')
